## Configuration
_Initial steps to get the notebook ready to play nice with our repository. Do not delete this section._

Code formatting with [black](https://pypi.org/project/nb-black/).

In [1]:
%load_ext lab_black

In [2]:
import os
import pytz
import glob
import pathlib

this_dir = pathlib.Path(os.path.abspath(""))
data_dir = this_dir / "data"

In [3]:
import requests
import pandas as pd
from datetime import datetime

## Download

Retrieve the page

In [4]:
url = "https://services2.arcgis.com/RETsakmE0SJfZXCd/arcgis/rest/services/Covid19_Cases_New_Workflow_Testing_FS/FeatureServer/0/query?f=json&where=1%3D1&returnGeometry=false&spatialRel=esriSpatialRelIntersects&outFields=*&groupByFieldsForStatistics=ReportingArea&orderByFields=ReportingArea%20asc&outStatistics=%5B%7B%22statisticType%22%3A%22count%22%2C%22onStatisticField%22%3A%22ReportingArea%22%2C%22outStatisticFieldName%22%3A%22count_result%22%7D%5D&resultType=standard&cacheHint=true"

In [5]:
r = requests.get(url)

In [6]:
data = r.json()

## Parse

In [7]:
dict_list = []

In [8]:
for item in data["features"]:
    d = dict(
        county="Yolo",
        area=item["attributes"]["ReportingArea"],
        confirmed_cases=item["attributes"]["count_result"],
    )
    dict_list.append(d)

KeyError: 'features'

In [9]:
df = pd.DataFrame(dict_list)

Get timestamp

In [10]:
date_url = "https://services2.arcgis.com/RETsakmE0SJfZXCd/arcgis/rest/services/Covid19_Cases_New_Workflow_Testing_FS/FeatureServer/0?f=json"
date_r = requests.get(date_url)
date_data = date_r.json()

In [11]:
timestamp = date_data["editingInfo"]["lastEditDate"]

In [12]:
timestamp = datetime.fromtimestamp((timestamp / 1000))

In [13]:
latest_date = pd.to_datetime(timestamp).date()

In [14]:
df["county_date"] = latest_date

## Vet

In [15]:
try:
    assert not len(df) > 5
except AssertionError:
    raise AssertionError("Yolo County's scraper has extra rows")

In [16]:
try:
    assert not len(df) < 5
except AssertionError:
    raise AssertionError("Yolo County's scraper is missing rows")

## Export

Set date

In [17]:
tz = pytz.timezone("America/Los_Angeles")

In [18]:
today = datetime.now(tz).date()

In [19]:
slug = "yolo"

In [20]:
df.to_csv(data_dir / slug / f"{today}.csv", index=False)

## Combine

In [21]:
csv_list = [
    i
    for i in glob.glob(str(data_dir / slug / "*.csv"))
    if not str(i).endswith("timeseries.csv")
]

In [22]:
df_list = []
for csv in csv_list:
    if "manual" in csv:
        df = pd.read_csv(csv, parse_dates=["date"])
    else:
        file_date = csv.split("/")[-1].replace(".csv", "")
        df = pd.read_csv(csv, parse_dates=["county_date"])
        df["date"] = file_date
    df_list.append(df)

In [23]:
df = pd.concat(df_list).sort_values(["date", "area"])

In [24]:
df.to_csv(data_dir / slug / "timeseries.csv", index=False)